# Export HadISD data

In [8]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
home_path = "/gws/nopw/j04/duicv/yuansun/"

In [7]:
start_time = pd.Timestamp('2022-01-01 00:00:00')
end_time = pd.Timestamp('2022-12-31 23:00:00')
station_name = 'hadisd'

# Observation

In [9]:
ds_obs = xr.open_dataset(f'{home_path}dap.ceda.ac.uk/badc/ukmo-hadobs/data/insitu/MOHC/HadOBS/HadISD/subdaily/HadISDTable/r1/v3-4-0-2023f/station_data/03334099999_HadISD_HadOBS_19310101-20240101_v3-4-0-2023f.nc')
ds_obs_sel = ds_obs.sel(time =slice(start_time, end_time))
ds_obs_sel

<xarray.Dataset> Size: 7MB
Dimensions:                      (coordinate_length: 1, time: 8726, test: 71)
Coordinates:
    longitude                    (coordinate_length) float64 8B ...
    latitude                     (coordinate_length) float64 8B ...
    elevation                    (coordinate_length) float64 8B ...
  * time                         (time) datetime64[ns] 70kB 2022-01-01 ... 20...
Dimensions without coordinates: coordinate_length, test
Data variables: (12/25)
    stationname                  |S12 12B ...
    station_identification_code  (time) object 70kB ...
    tas                          (time) float64 70kB ...
    tds                          (time) float64 70kB ...
    psl                          (time) float64 70kB ...
    stnlp                        (time) float64 70kB ...
    ...                           ...
    precip18_depth               (time) float64 70kB ...
    precip24_depth               (time) float64 70kB ...
    cbase                        (time) float64 70kB ...
    wg                           (time) float64 70kB ...
    pastsigwx1                   (time) float64 70kB ...
    quality_control_flags        (time, test) float64 5MB ...
Attributes: (12/39)
    title:                       HadISD
    institution:                 Met Office Hadley Centre, Exeter, UK
    source:                      HadISD data product
    references:                  Dunn, 2019, Met Office Hadley Centre Technic...
    creator_name:                Robert Dunn
    creator_url:                 www.metoffice.gov.uk
    ...                          ...
    station_information:         Where station is a composite the station id ...
    Conventions:                 CF-1.6
    Metadata_Conventions:        Unidata Dataset Discovery v1.0, CF Discrete ...
    featureType:                 timeSeries
    processing_date:             09-Jan-2024
    history:                     Created by mk_netcdf_files.py \nDuplicate Mo...

In [20]:
np.unique(ds_obs_sel['precip18_depth'])

array([nan])

In [5]:
lat = ds_obs_sel['latitude'][0].values
lon = ds_obs_sel['longitude'][0].values
print(lat, lon)

53.354 -2.275


In [7]:
def compute_relative_humidity(tas, tds):
    numerator = 17.62 * (tds - tas)
    denominator = 243.12 + tds
    rh = 100 * np.exp(numerator / denominator)
    return rh.clip(0, 100)
ds_obs_tas = ds_obs_sel['tas']
ds_obs_tds = ds_obs_sel['tds']
ds_obs_rh = compute_relative_humidity(ds_obs_tas, ds_obs_tds)
ds_obs_rh

<xarray.DataArray (time: 8726)> Size: 70kB
array([81.15301375, 87.05112103, 87.05112103, ..., 93.22393743,
       93.19778918, 93.19778918])
Coordinates:
  * time     (time) datetime64[ns] 70kB 2022-01-01 ... 2022-12-31T23:00:00

In [11]:
df_rh = ds_obs_rh.to_dataframe(name='RH').reset_index()
df_tas = ds_obs_tas.to_dataframe(name='TAS').reset_index()
df_wind = ds_obs_sel['ws'].to_dataframe(name='WIND').reset_index()

# Merge on shared coordinates (e.g. time, lat, lon)
df_obs = pd.merge(df_tas, df_rh, on=['time']).merge(df_wind, on=['time'])
df_obs.loc[df_obs['WIND'] < -1e10, 'WIND'] = np.nan
df_obs.to_csv('./data_for_figure/hadisd_obs.csv', index=False)
df_obs

,time,TAS,RH,WIND
0,2022-01-01 00:00:00,13.0,81.153014,2.6
1,2022-01-01 01:00:00,13.0,87.051121,3.6
2,2022-01-01 02:00:00,13.0,87.051121,3.6
3,2022-01-01 03:00:00,13.0,87.051121,2.6
4,2022-01-01 04:00:00,13.0,81.153014,4.1
...,...,...,...,...
8721,2022-12-31 19:00:00,10.0,93.249886,2.1
8722,2022-12-31 20:00:00,9.0,100.000000,2.6
8723,2022-12-31 21:00:00,9.0,93.223937,3.1
8724,2022-12-31 22:00:00,8.0,93.197789,2.6


In [10]:
df_obs['WIND'].min(), df_obs['WIND'].max()

(-2e+30, 17.5)

In [4]:
df_check = pd.read_csv('./data_for_figure/hadisd_obs.csv')
df_check['TAS'].max(), df_check['RH'].max(), df_check['WIND'].max()

(np.float64(37.0), np.float64(100.0), np.float64(17.5))

# GM_NOAH_SLUCM

In [9]:
ds_sim_sample = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_wrf/wrfout_d01_2021-12-25_00:00:00.nc')
model_lat = ds_sim_sample['XLAT'][0, :, 0].values
model_lon = ds_sim_sample['XLONG'][0, 0, :].values
lat_idx = np.abs(model_lat - lat).argmin()
lon_idx = np.abs(model_lon - lon).argmin()
print(lat_idx, lon_idx)

In [ ]:
lat_idx = 7
lon_idx = 28